In [62]:
from rtt_tools import dump_data
from rtt_tools.dump_data import *  # pussy died because of this wildcard import
from rtt_tools.export import *
import json 

logger = logging.getLogger(__name__)
coloredlogs.CHROOT_FILES = []
coloredlogs.install(level=logging.INFO, use_chroot=False)

In [73]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

from rtt_tools.gen.max_rounds import FUNC_DB, FuncDb, FuncInfo
loader = dump_data.Loader()

In [74]:
import json
js1 = json.load(open('/tmp/rtt-results-aesref-01.json'))

In [75]:
js = js1
testvariant = {}
datatype = {}
test_db = collections.defaultdict(lambda: 0)
test_db_inv = collections.defaultdict(lambda: 0)
test_db_ctr = set()
tstart = time.time()
num_exps = len(js)

# test-desc -> summary + details
refres = collections.defaultdict(              # test-desc
    lambda: collections.OrderedDict([        
        ('num_rejects', 0),
        ('num_total', 0),
        ('runs', collections.defaultdict(lambda: dict())),
]))

# experiment id -> tests failed in this round
refrounds = collections.defaultdict(           # seed
    lambda: collections.OrderedDict([        
        ('total', 0),
        ('rejects', []),
]))

for rix, rec in enumerate(js):
    data_type = rec['data_type']
    exp_info = loader.break_exp_ph4_aes_ref(data_type)
    if not exp_info or not exp_info.fnc:
        print('Could not parse %s' % data_type)
        continue
    
    seed_int = int(exp_info.seed, 16)    
    #if seed_int >= 2000:
    #    continue

    test_desc = f'{rec["test"]}##[{rec["variant_type"]}][{rec["variant"]}]##[{rec["subtest_type"]}][{rec["subtest"]}]'
    refrec = refres[test_desc]
    testvariant[test_desc] = collections.OrderedDict([        
        ('variant_type', rec["variant_type"]),
        ('variant', rec["variant"]),
        ('variant_id', rec["variant_id"]),
        ('subtest_type', rec["subtest_type"]),
        ('subtest', rec["subtest"]),
        ('exid', rec["exid"]),
    ])
    
    # iterate over subtests
    nsubs_passed = 0
    for sub in rec['subs']:
        is_passed = sub['stats'][0]['pass']
        nsubs_passed += int(is_passed)
        
    nsubs = len(rec['subs'])
    npass_ratio = nsubs_passed / nsubs
    variant_passed = npass_ratio >= 0.99  # tune this ratio! Ratio of passed subtests to consider test passed
    
    refrec['num_rejects'] += int(not variant_passed)
    refrec['num_total'] += 1
    refrec['runs'][seed_int] = collections.OrderedDict([
        ('recidx', rix),
        ('nsubs', nsubs),
        ('nsubsp', nsubs_passed),
        ('nsubsr', npass_ratio),
        ('passed', variant_passed)
    ])
    
    refrounds[seed_int]['total'] += 1
    if not variant_passed:
        refrounds[seed_int]['rejects'].append((test_desc, rix))
    # print('#: %s, pass: %s, pct: %2.7f' % (nsubs, nsubs_passed, 100*nsubs_passed/nsubs))
    
# Reindex tests so we have stable IDs
all_tests = sorted(list(test_db_ctr))
for ix, test in enumerate(all_tests):
    test_db[test] = ix
    test_db_inv[ix] = test
    
#print(funrounds, funtype, funcres)
#print(json.dumps(funcres, indent=2))
print('Proc1 done in %s s' % (time.time() - tstart))

Proc1 done in 88.37884473800659 s


In [76]:
kspaces = [20, 38, 20, 20, 20, 20, 20]
krec = re.compile(r'^\[(.*?)\]\s*\[(.*?)\]$')
test_keys = sorted(list(refres.keys()))

rej_db = {}
print('Table of test rejections')
for ctest in test_keys:
    rec = refres[ctest]
    ctest2 = ctest.replace('(Minimum Distance)', '(MD)')
    ctest2 = ctest2.replace('Minimum Distance', 'MinDist')
    ctest2 = ctest2.replace('NIST Statistical Testing Suite', 'NIST-STS')
    ctest2 = ctest2.replace('TestU01 Block Alphabit', 'TestU01 BAlp')    
    ctest2 = ctest2.replace('Discrete Fourier Transform (Spectral) Test', 'D. Fourier T.')    
    ctest2 = ctest2.replace('Template Matching Test', 'TplMT')    
    
    kcomp = ctest2.split("|", 1)
    kcomp = [kcomp[0]] + kcomp[1].split('##')
    kcompr = kcomp[:2]
    for ixx, cxx in enumerate(kcomp[2:]):
        m = krec.match(cxx)
        if not m:
            kcompr.append(cxx)
            continue
        # kcompr.append(m.group(1))
        kcompr.append(m.group(2))
        
    kform = ' | '.join([x.ljust(kspaces[ix] if ix < len(kspaces) else 20) for ix, x in enumerate(kcompr)])
    rej_rate = (rec["num_rejects"] / rec["num_total"])
    rej_db[ctest] = rej_rate

    print(f'R: {"%3d" % rec["num_rejects"]} / {"%4d" % rec["num_total"]} ({"%.6f" % rej_rate} %) : {kform}')
    
#print(json.dumps(rej_db, indent=2))
for ckey in list(rej_db.keys()):
    diff = abs(rej_db[ckey] - rej_db_old[ckey])
    print(f' diff: {"%0.8f" % diff} : {ckey}')
#rej_db_old = rej_db
#rej_db_2k = rej_db
#rej_db_7k = rej_db_old

Table of test rejections
R: 106 / 10079 (0.010517 %) : Dieharder            | DAB DCT                                | 1                    |                     
R:  46 / 10079 (0.004564 %) : Dieharder            | Diehard 32x32 Binary Rank Test         | 19                   |                     
R:  61 / 10079 (0.006052 %) : Dieharder            | Diehard 3d Sphere (MD) Test            | 100                  |                     
R:  35 / 10079 (0.003473 %) : Dieharder            | Diehard 6x8 Binary Rank Test           | 41                   |                     
R:  62 / 10079 (0.006151 %) : Dieharder            | Diehard Birthdays Test                 | 100                  |                     
R:  67 / 10079 (0.006647 %) : Dieharder            | Diehard Bitstream Test                 | 95                   |                     
R:  63 / 10079 (0.006251 %) : Dieharder            | Diehard Count the 1s (stream) Test     | 100                  |                     
R:  34 / 

In [77]:
fails_ctr = collections.Counter()
for seed in sorted(list(refrounds.keys())):
    rec = refrounds[seed]
    total = rec["total"]
    num_rej = len(rec["rejects"])
    fail_ratio = ((num_rej / total) if total > 0 else -1)

    if total != 245:
        continue

    if num_rej > 4: print(f'S: {"%8d" % seed}, R: {"%04d" % num_rej} / {"%04d" % total}, {"%.5f" % fail_ratio}')
    fails_ctr[num_rej] += 1

print('Number of test rejections per seed run')
for ix in sorted(list(fails_ctr.keys())):
    print(f' {"%3d" % ix} : {"%5d" % fails_ctr[ix]}')

S:      302, R: 0005 / 0245, 0.02041
S:      709, R: 0005 / 0245, 0.02041
S:      774, R: 0006 / 0245, 0.02449
S:      897, R: 0013 / 0245, 0.05306
S:     1191, R: 0007 / 0245, 0.02857
S:     1337, R: 0005 / 0245, 0.02041
S:     1358, R: 0006 / 0245, 0.02449
S:     1607, R: 0005 / 0245, 0.02041
S:     1624, R: 0007 / 0245, 0.02857
S:     1981, R: 0005 / 0245, 0.02041
S:     2087, R: 0005 / 0245, 0.02041
S:     2379, R: 0006 / 0245, 0.02449
S:     3125, R: 0005 / 0245, 0.02041
S:     3327, R: 0006 / 0245, 0.02449
S:     4316, R: 0005 / 0245, 0.02041
S:     4335, R: 0005 / 0245, 0.02041
S:     4804, R: 0005 / 0245, 0.02041
S:     4842, R: 0006 / 0245, 0.02449
S:     5207, R: 0006 / 0245, 0.02449
S:     5356, R: 0005 / 0245, 0.02041
S:     5610, R: 0005 / 0245, 0.02041
S:     6233, R: 0005 / 0245, 0.02041
S:     6327, R: 0005 / 0245, 0.02041
S:     7668, R: 0076 / 0245, 0.31020
S:     7772, R: 0006 / 0245, 0.02449
S:     7774, R: 0005 / 0245, 0.02041
S:     8318, R: 0007 / 0245, 0.02857
S

In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))